In [1]:
# http://www.pytorch123.com/

In [3]:
from __future__ import print_function
import torch

In [13]:
x = torch.empty(5, 3)
print(x.size())
x

torch.Size([5, 3])


tensor([[6.4460e-44, 1.6255e-43, 1.4153e-43],
        [1.5414e-43, 1.6115e-43, 1.5554e-43],
        [1.5975e-43, 5.6052e-44, 1.2752e-43],
        [7.4269e-44, 6.4460e-44, 7.4269e-44],
        [6.1657e-44, 4.4842e-44, 7.1466e-44]])

In [5]:
torch.rand(5, 3)

tensor([[0.9143, 0.6363, 0.1054],
        [0.7578, 0.6292, 0.4246],
        [0.0159, 0.1496, 0.8645],
        [0.1217, 0.0168, 0.7807],
        [0.3239, 0.8942, 0.2614]])

In [ ]:
vec = torch.rand(5)
torch.max(vec,1)

In [9]:
# 构造tensor
x = torch.tensor([5.5, 3, 9.0])
x

tensor([5.5000, 3.0000, 9.0000])

In [10]:
x.size()

torch.Size([3])

In [11]:
torch.tensor([[5.5, 3, 9.0],[6.0, 9.0, 5.0]]).size()

torch.Size([2, 3])

In [14]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8) # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [19]:
x.item()

ValueError: only one element tensors can be converted to Python scalars

In [17]:
torch.randn(1).item()

0.9882491827011108

### 自动微分 auto_grad
x --> y --> z --> out

x.grad <-- backward -- out

In [33]:
x = torch.ones(2, 2, requires_grad=True)  # requires_grad=True 来跟踪与它相关的计算
y = x + 2
print(y)
y.grad_fn

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [34]:
z = y * y * 3
out = z.mean()
print(z)
out

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)


tensor(27., grad_fn=<MeanBackward0>)

In [35]:
# out.backward()
# x.grad

In [36]:
out.backward(torch.tensor(2.0))
x.grad

tensor([[9., 9.],
        [9., 9.]])

### 神经网络

In [39]:
import torch.nn as nn
import torch.nn.functional as F

In [108]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        # Max pooling over a (2, 2) window
        # x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.relu(self.conv1(x))  # torch.Size([1, 6, 28, 28]) :32-5+1
        x = F.max_pool2d(x,(2,2))  # torch.Size([1, 6, 14, 14]) : 28/2
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [116]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [117]:
params = list(net.parameters())

In [118]:
params[0].size()

torch.Size([6, 1, 5, 5])

In [119]:
input = torch.randn(1, 1, 32, 32) # 32*32的输入
out = net(input)
out.size()                

torch.Size([1, 10])

In [120]:
input

tensor([[[[ 1.8508e-01,  1.1058e+00, -9.2410e-04,  ..., -9.2500e-01,
           -1.6046e+00, -9.1191e-01],
          [ 1.5073e+00,  7.2835e-01,  6.1985e-01,  ...,  1.8675e+00,
            6.5220e-01,  1.0047e-01],
          [-6.1039e-01, -1.2427e-01,  7.4717e-01,  ..., -2.0162e-01,
           -5.1271e-01,  3.5293e-01],
          ...,
          [ 9.6776e-02, -1.5472e+00,  2.9112e-02,  ..., -1.6700e-01,
            4.2853e-01, -3.3378e-01],
          [-4.3333e-01, -5.7990e-02, -1.1173e+00,  ..., -3.1305e-02,
            2.4031e-01, -2.6599e+00],
          [ 1.9464e+00,  4.0971e-01,  1.2905e+00,  ..., -2.6088e+00,
           -1.3387e-01,  9.0829e-01]]]])

In [121]:
net.zero_grad()  # 把所有参数梯度缓存器置零
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
out.backward(torch.randn(1, 10))
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0267, -0.0248,  0.0265, -0.0137,  0.0466,  0.0197])


In [93]:
# input

In [131]:
# net = Net()
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()  # 损失函数
loss = criterion(output, target)
loss

tensor(0.6880, grad_fn=<MseLossBackward>)

In [132]:
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward() # 将损失反向传播
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0060,  0.0002,  0.0031,  0.0009,  0.0030, -0.0011])


In [139]:
# 参数更新规则：weight = weight - learning_rate * gradient
# 参数更新：torch.optim
# 使用不同的更新规则，类似于 SGD, Nesterov-SGD, Adam, RMSProp, 等

In [137]:
import torch.optim as optim
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update

In [135]:
dd = torch.rand(5, 3)
dd

tensor([[0.5375, 0.1636, 0.2614],
        [0.3691, 0.7959, 0.0344],
        [0.4400, 0.2856, 0.5487],
        [0.8095, 0.5042, 0.8373],
        [0.2461, 0.1447, 0.1760]])

In [136]:
dd.sub_(torch.ones_like(dd))

tensor([[-0.4625, -0.8364, -0.7386],
        [-0.6309, -0.2041, -0.9656],
        [-0.5600, -0.7144, -0.4513],
        [-0.1905, -0.4958, -0.1627],
        [-0.7539, -0.8553, -0.8240]])